In [ ]:
import numpy as np
import sympy as sp
from numpy.linalg import inv
from numpy import matmul as mul

import time
import matplotlib.pyplot as plt

# Printing Matrices Nicely ______________________________________________________________________________
def matprint(mat, fmt="g"):
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("")
        
# Surface Green's Function Calculations _________________________________________________________________
def surface_greens_func_calc_general(Energy,z,n_atoms,Periodicity,N_periods,oneD_test,Gamma,Beta,potential,error_output,print_output):
    # Calculates surface green's function in general with entirely random input matrices to construct Hamiltonian

    # Energy - input energy in eV
    # z - imaginary componenet added to energy, usually between e-3 and e-8
    # n_atoms - number of atoms per layer
    # Periodicity - periodicity, after how many layers is there repition
    # N_periods - total number of periods in Hamiltonian
    # oneD_test (T/F) - Testing 1 dimensional case with set tau structure
    # Gamma - value of Gamma in 1d test case
    # Beta - value of Beta in 1d test case
    # potential - value of potential in 1d test case
    # error_output - output error instead of green's surface functions
    # print_output - print ressults for different methods
    
    # OUTPUT - array containing surface green's functions calculated in different methods.
    
    E = Energy+z*1j

    n = n_atoms # Atoms per layer
    P = Periodicity # Periodicity
    if oneD_test == True:
        n = 1
        P = 2
    N = N_periods # Number of periods

    # Create Hamiltonian
    alpha = np.zeros((n,n,P),dtype=complex)
    tau = np.zeros((n,n,P),dtype=complex)
    if oneD_test == False:
        for i in range(P):
            alpha[:,:,i] = (np.random.random((n,n)))+1j*(np.random.random((n,n)))
            tau[:,:,i] = (np.random.random((n,n)))+1j*(np.random.random((n,n)))
    else:
        for i in range(P):
            alpha[:,:,i] = np.diag(potential*np.ones(n))
            if i%2 == 0:
                tau[:,:,i] = np.diag(Beta*np.ones(n))
            else:
                tau[:,:,i] = np.diag(Gamma*np.ones(n))

    H = np.zeros((N*P*n,N*P*n),dtype=complex)
    for i in range(N):
        for k in range(P):
            H[i*P*n+k*n:i*P*n+(k+1)*n,i*P*n+k*n:i*P*n+(k+1)*n] = alpha[:,:,k]

            if i*P+(k+1)<P*N:
                H[i*P*n+k*n:i*P*n+(k+1)*n,i*P*n+(k+1)*n:i*P*n+(k+2)*n] = tau[:,:,k]
                H[i*P*n+(k+1)*n:i*P*n+(k+2)*n,i*P*n+k*n:i*P*n+(k+1)*n] = tau[:,:,k].T.conj()

    # Condensation Algorithm ##################################################################
    t0_condensation = time.time()
    
    # Recursive inversion of C matrix
    H_tilde = np.zeros((n,n,P,P),dtype=complex)
    H_tilde[:,:,P-1,P-1] = inv(E*np.diag(np.ones(n))-alpha[:,:,P-1])

    for i in range(P-2,0,-1):
        H_tilde[:,:,i,i] = inv(E*np.diag(np.ones(n))-alpha[:,:,i]-mul(tau[:,:,i],mul(H_tilde[:,:,i+1,i+1],tau[:,:,i].T.conj())))
        H_tilde[:,:,i,P-1] = mul(H_tilde[:,:,i,i],mul(tau[:,:,i],H_tilde[:,:,i+1,P-1]))

    C_tilde = np.zeros((n,n,P),dtype=complex)
    C_tilde[:,:,1] = H_tilde[:,:,1,1]

    for i in range(2,P):
        C_tilde[:,:,i] = H_tilde[:,:,i,i]+mul(H_tilde[:,:,i,i],mul(mul(tau[:,:,i-1].T.conj(),mul(C_tilde[:,:,i-1],tau[:,:,i-1])),H_tilde[:,:,i,i]))  
        
    # Makes Condensation Matices
    Xi_s = alpha[:,:,0]+mul(tau[:,:,0],mul(H_tilde[:,:,1,1],tau[:,:,0].T.conj()))
    Xi = Xi_s+mul(tau[:,:,P-1],mul(C_tilde[:,:,P-1],tau[:,:,P-1].T.conj()))
    Pi = mul(tau[:,:,0],mul(H_tilde[:,:,1,P-1],tau[:,:,P-1]))

    # Condensed Matrix    
    H_conds = np.zeros((N*n,N*n),dtype=complex)
    for i in range(N):
        if i == 0:
            H_conds[i*n:(i+1)*n,i*n:(i+1)*n] = Xi_s
        else:
            H_conds[i*n:(i+1)*n,i*n:(i+1)*n] = Xi
            H_conds[(i-1)*n:i*n,i*n:(i+1)*n] = Pi
            H_conds[i*n:(i+1)*n,(i-1)*n:i*n] = Pi.T.conj()
            
    t1_condensation = time.time()

    # Eigenvalue Formulation ##################################################################
    # Eigenvalue Matrices
    t0_eig = time.time()
    
    A = np.zeros((2*n,2*n),dtype=complex)
    A[0:n,n:2*n] = np.diag(np.ones(n))
    A[n:2*n,0:n] = -Pi.T.conj()
    A[n:2*n,n:2*n] = E*np.diag(np.ones(n))-Xi

    B = np.zeros((2*n,2*n),dtype=complex)
    B[0:n,0:n] = np.diag(np.ones(n))
    B[n:2*n,n:2*n] = Pi

    # Selcting eigenvalues with mag of less than one
    U = np.linalg.eig(mul(inv(B),A)).eigenvectors
    Lambda = np.linalg.eig(mul(inv(B),A)).eigenvalues
    
    if oneD_test == True:
        indices = np.where(Lambda == min(Lambda))
        U = U[0:n,indices[0]]
        Lambda = np.diag(Lambda[indices[0]])
    else:
        indices = np.where(np.abs(Lambda) < 1)
        index = indices[0]
        
        # remove extra vectors
        if len(index) > n:
            index = index[0:n]  
        
        U = U[0:n,index]
        Lambda = np.diag(Lambda[index])

    # Pseudo invert U
    Q,R = np.linalg.qr(U)
    if R.shape[0] != R.shape[1]:
        Q,R = np.linalg.qr(U)

    U_tilde = mul(inv(R),Q.T.conj())

    # Self Energy Matrix
    F = mul(U,mul(Lambda,U_tilde))
    Sigma_dash = mul(Pi,F)

    # Green's function
    g_00_eig = inv(E*np.diag(np.ones(n))-Xi_s-Sigma_dash)
    
    t1_eig = time.time()
    
    
    
    print(Sigma_dash)
    
    
    

    # Decimation Formulation ##################################################################
    # Initial matrices
    t0_decimation = time.time()
    
    mu_0 = E*np.diag(np.ones(n))-Xi_s
    nu_0 = E*np.diag(np.ones(n))-Xi
    gamma_0 = Pi

    mu_old = mu_0-mul(gamma_0,mul(inv(nu_0),gamma_0.T.conj()))
    nu_old = nu_0-mul(gamma_0,mul(inv(nu_0),gamma_0.T.conj()))-mul(gamma_0.T.conj(),mul(inv(nu_0),gamma_0))
    gamma_old = mul(gamma_0,mul(inv(nu_0),gamma_0))
    zeta_old = mul(gamma_0.T.conj(),mul(inv(nu_0),gamma_0.T.conj()))

    # Iterative Procedure
    iterations = 0
    error = 1
    while(error>10**-8):
        mu_new = mu_old-mul(gamma_old,mul(inv(nu_old),gamma_old.T.conj()))
        nu_new = nu_old-mul(gamma_old,mul(inv(nu_old),gamma_old.T.conj()))-mul(gamma_old.T.conj(),mul(inv(nu_old),gamma_old))
        gamma_new = mul(gamma_old,mul(inv(nu_old),gamma_old))
        zeta_new = mul(gamma_old.T.conj(),mul(inv(nu_old),gamma_old.T.conj()))

        error = abs(np.sum(mu_new-mu_old))
        
        mu_old = mu_new
        nu_old = nu_new
        gamma_old = gamma_new
        zeta_old = zeta_new
        
        iterations += 1
        
    g_00_dec = inv(mu_new)
    
    t1_decimation = time.time()

    # Recursive (Datta) Formulation ##################################################################
    if oneD_test == True:
        g_old = (E-Gamma**2)**(-1)

        for i in range(iterations):
            if i%2 == 0:
                g_new = (E-Gamma**2*g_old)**(-1)
            else:
                g_new = (E-Beta**2*g_old)**(-1)
            g_old = g_new

        g_00_rec = g_new

    # Direct Formulation ##################################################################
    t2_condensation = time.time()
    g_surf_conds = inv(E*np.diag(np.ones(N*n))-H_conds)
    g_surf_conds_dir = g_surf_conds[0:n,0:n]
    t3_condensation = time.time()    

    t0_direct = time.time()
    g_surf = inv(E*np.diag(np.ones(N*P*n))-H)
    g_surf_dir = g_surf[0:n,0:n]
    t1_direct = time.time()

    if oneD_test == True:
        g_quadratic = 1/(2*Gamma**2*(E-potential))*((E-potential)**2+Gamma**2-Beta*np.conj(Beta)-np.sqrt(((E-potential)**2+Gamma**2-Beta*np.conj(Beta))**2-4*Gamma**2*(E-potential)**2))    
    
    if print_output == True:
        print("Eigenvalue Method")
        #matprint(g_00_eig)
        print("Error from Cond")
        print(np.sum(abs(g_00_eig-g_surf_conds_dir))/n**2)
        print("Error from Direct")
        print(np.sum(abs(g_00_eig-g_surf_dir))/n**2)
        print("Time")
        print(t1_condensation-t0_condensation+t1_eig-t0_eig)
        print("___________________________")
        print("Decimation Method")
        #matprint(g_00_dec)
        print("# Iterations before epsilon < 10^-8")
        print(iterations)
        print("Error from Cond")
        print(np.sum(abs(g_00_dec-g_surf_conds_dir))/n**2)
        print("Error from Direct")
        print(np.sum(abs(g_00_dec-g_surf_dir))/n**2)   
        print("Time")
        print(t1_condensation-t0_condensation+t1_decimation-t0_decimation)
        print("___________________________")
        if oneD_test == True:
            print("Recursive Relation Datta")
            print(g_00_rec)
            print("Error from Direct")
            print(np.sum(abs(g_00_rec-g_surf_dir))/n**2)
            print("___________________________")
        print("Direct Condensed Method")
        #matprint(g_surf_conds_dir)
        print("Error from Direct")
        print(np.sum(abs(g_surf_conds_dir-g_surf_dir))/n**2)
        print("Time")
        print(t1_condensation-t0_condensation+t3_condensation-t2_condensation)
        print("___________________________")
        print("Direct Method")
        #matprint(g_surf_dir)
        print("Time")
        print(t1_direct-t0_direct)
        print("___________________________")
        if oneD_test == True:
            print("Quadratic Formulation")
            print(g_quadratic)
    
    if error_output == False:
        if oneD_test == False:    
            return([g_00_eig[0:n,0:n],g_00_dec[0:n,0:n],g_surf_conds_dir[0:n,0:n],g_surf_dir[0:n,0:n]])  
        else:
            return([g_00_eig[0,0],g_00_dec[0,0],g_00_rec,g_quadratic,g_surf_conds_dir[0,0],g_surf_dir[0,0]])
    else:
        return([np.sum(abs(g_00_eig-g_surf_conds_dir))/n**2,np.sum(abs(g_00_eig-g_surf_dir))/n**2,iterations,np.sum(abs(g_00_dec-g_surf_conds_dir))/n**2,np.sum(abs(g_00_dec-g_surf_dir))/n**2,np.sum(abs(g_surf_conds_dir-g_surf_dir))/n**2])  

In [ ]:
# Error Calculation and Convergence Study
nsteps = 300

error_array_n = np.zeros((6,nsteps+1))
error_array_p = np.zeros((6,nsteps+1))
error_array_N = np.zeros((6,nsteps+1))

for i in range(3,nsteps+1):
    #error_array_p[:,i] = surface_greens_func_calc_general(-4,10**(-i),17,4,2,False,1,1.1,True,False)
    error_array_N[:,i] = surface_greens_func_calc_general(-4,10**-8,2,2,i,False,1,1.1,0,True,False)

In [ ]:
# Plotting 
fig, axs = plt.subplots(6, 1, figsize=(10, 36), sharex=True)

axs[0].plot(error_array_p[2,:],'o',label="Eigenvalue Method to Condensed Error")
axs[1].plot(error_array_p[3,:],'o',label="Decimation Method to Condensed Error")
axs[2].plot(error_array_p[4,:],'o',label="Condensed Method to Direct Error")

axs[3].plot(error_array_N[0,:],'o',label="Eigenvalue Method to Condensed Error")
axs[4].plot(error_array_N[3,:],'o',label="Decimation Method to Condensed Error")
axs[5].plot(error_array_N[5,:],'o',label="Condensed Method to Direct Error")



for i in range(0,3):
    axs[i].set_ylabel('Error')
    axs[i].set_xlabel('p value')
    axs[i].legend()
    
for i in range(3,6):
    axs[i].set_ylabel('Error')
    axs[i].set_xlabel('N value')
    axs[i].legend()    

axs[0].set_title('Eigenvector Method')
axs[1].set_title('Decimation Method')
axs[2].set_title('Condensed Method')

plt.tight_layout()
plt.show()    


In [ ]:
surface_greens_func_calc_general(-2,10**-8,17,8,4,True,2.5,2.2287,0,False,True)

In [ ]:
matprint(Q)
matprint(R)
matprint(mul(inv(R),Q.T.conj()))

In [ ]:
# Test over several E values

Emin = -10
Emax = 0
stepsize = 0.01
Layers_probe = 40

gamma_value = 2.5
beta_value = 2

ressults = np.zeros((int((Emax-Emin)/stepsize)+1,6),dtype=complex)
greens_func = np.zeros((int((Emax-Emin)/stepsize)+1,6,2*Layers_device,2*Layers_device),dtype=complex)

for i, E in enumerate(np.arange(Emin, Emax, stepsize)):
    ressults[i,:] = surface_greens_func_calc_general(E,10**-8,4,3,Layers_probe,True,gamma_value,beta_value,False,False)
    for k in range(6):
        greens_func[i,k,:,:] = Greens_1D(E, gamma_value, beta_value, ressults[i,k], Layers_device)

In [ ]:
# Plotting 

x_values = np.linspace(Emin, Emax, int((Emax - Emin) / stepsize) + 1)

fig, axs = plt.subplots(4, 1, figsize=(10, 10), sharex=True)
axs[0].plot(x_values, abs(abs(ressults[:, 1])), label="Decimation Method")
axs[1].plot(x_values, abs(abs(ressults[:, 0])), label="Eigenvector Method")
axs[2].plot(x_values, abs(abs(ressults[:, 2])), label="Recursion Method")
axs[3].plot(x_values, abs(abs(ressults[:, 4])), label="H Direct Method")

# Plotting the quadratic case in each subplot
for ax in axs:
    ax.plot(x_values, abs(abs(ressults[:, 3])), label="Quadratic", linestyle='--', color='gray')

# Adding vertical lines
for ax in axs:
    ax.axvline(x=-(beta_value + gamma_value), color='b', linestyle='--')
    ax.axvline(x=-(-beta_value + gamma_value), color='b', linestyle='--')

axs[3].set_xlabel('E (ev)')
for ax in axs:
    ax.set_ylabel('|g|')
    ax.set_xlim(Emin, Emax)
    ax.set_ylim(-0.1, 3)
    ax.legend()

axs[0].set_title('Decimation Method vs Quadratic')
axs[1].set_title('Eigenvector Method vs Quadratic')
axs[2].set_title('Recursive Method vs Quadratic')
axs[3].set_title('H Direct Method vs Quadratic')

plt.tight_layout()
plt.show()

In [ ]:
# Inversion of C matrix recursively
C = H[n:P*n,n:P*n]
C_tilde = inv(E*np.diag(np.ones((P-1)*n))-C)

H_tilde = np.zeros((n,n,P,P),dtype=complex)
H_tilde[:,:,P-1,P-1] = inv(E*np.diag(np.ones(n))-alpha[:,:,P-1])

for i in range(P-2,0,-1):
    H_tilde[:,:,i,i] = inv(E*np.diag(np.ones(n))-alpha[:,:,i]-mul(tau[:,:,i],mul(H_tilde[:,:,i+1,i+1],tau[:,:,i].T.conj())))
    H_tilde[:,:,i,P-1] = mul(H_tilde[:,:,i,i],mul(tau[:,:,i],H_tilde[:,:,i+1,P-1]))

C_find_tilde = np.zeros((n,n,P),dtype=complex)
C_find_tilde[:,:,1] = H_tilde[:,:,1,1]

for i in range(2,P):
    C_find_tilde[:,:,i] = H_tilde[:,:,i,i]+mul(H_tilde[:,:,i,i],mul(mul(tau[:,:,i-1].T.conj(),mul(C_find_tilde[:,:,i-1],tau[:,:,i-1])),H_tilde[:,:,i,i]))  

print("Top left elements")
matprint(H_tilde[:,:,1,1])
matprint(C_tilde[0:n,0:n])
print("_________________________________")
print("Top right elements")
matprint(C_tilde[0:n,(P-2)*n:(P-1)*n])
matprint(H_tilde[:,:,1,P-1])
print("Bottom right elements")
print("_________________________________") 
matprint(C_find_tilde[:,:,P-1])
matprint(C_tilde[(P-2)*n:(P-1)*n,(P-2)*n:(P-1)*n])


In [ ]:
theta = np.zeros((n,n,P),dtype=complex)
theta[:,:,0] = np.diag(np.ones(n))
theta[:,:,1] = mul(inv(tau[:,:,1]),alpha[:,:,1])

for i in range(2,P-1):
    theta[:,:,i] = mul(inv(tau[:,:,i]),mul(alpha[:,:,i],theta[:,:,i-1]))-mul(inv(tau[:,:,i]),mul(tau[:,:,i-1].T.conj(),theta[:,:,i-2],))

        


phi = np.zeros((n,n,P+1),dtype=complex)
phi[:,:,P] = np.diag(np.ones(n))
phi[:,:,P-1] = mul(inv(tau[:,:,P-2].T.conj()),alpha[:,:,P-1])
    
for i in range(P-2,1,-1):
    phi[:,:,i] = mul(inv(tau[:,:,i-1].T.conj()),mul(alpha[:,:,i],phi[:,:,i+1]))-mul(inv(tau[:,:,i-1].T.conj()),mul(tau[:,:,i],phi[:,:,i+2]))
      

C_tilde_11 = inv(alpha[:,:,1]-mul(tau[:,:,1],mul(phi[:,:,3],inv(phi[:,:,2]))))
C_tilde_nn = inv(alpha[:,:,P-2]-mul(tau[:,:,P-1].T.conj(),mul(theta[:,:,P-3],inv(theta[:,:,P-2]))))

matprint(C_tilde_11)
        
C_tilde_real = inv(C)
    
matprint(C_tilde_real[0:n,0:n])
matprint(C_tilde_real)